# PNR data fit

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import csv
import os
import glob
import math
from operator import add
from numpy import exp, loadtxt, pi, sqrt
from lmfit import Model

In [2]:
# import data
file = open(r"C:\\Users\\PC\\Desktop\\Ph.D\\PNR\\PNR_highpwr.txt", 'r')
file_string = np.array([row.strip().split(" ") for row in file][0:], dtype=float)
data_2D = file_string

In [3]:
x=np.arange(0,data_2D.shape[0],1)
y=np.arange(0,data_2D.shape[1],1)
X, Y = np.meshgrid(x, y)
# plt.plot(X, Y, marker='.', color='k', linestyle='none')

### Plot raw data

In [6]:
%matplotlib qt5

#color map

fig, ax = plt.subplots()
ax.set_xlabel(r'$\tau_{fal}$ (ns)')
ax.set_ylabel(r'$\tau_{ris}$ (ns)')
img = ax.imshow(data_2D)
fig.colorbar(img)

# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# ax.plot_surface(X, Y, data_2D, color='red', antialiased=True)

### Surface fit

In [5]:
# Defining the model function (sum of two Gaussians)

def double_gaussian(x, y, amp_1, cen_1_x,cen_1_y, wid_1_x, wid_1_y, amp_2, cen_2_x,cen_2_y, wid_2_x, wid_2_y):    
    return amp_1 * exp(-(x-cen_1_x)**2 / (2*wid_1_x**2))*exp(-(y-cen_1_y)**2 / (2*wid_1_y**2)) +\
           amp_2 * exp(-(x-cen_2_x)**2 / (2*wid_2_x**2))*exp(-(y-cen_2_y)**2 / (2*wid_2_y**2))


gmodel = Model(double_gaussian, independent_vars= ['x','y'],
               param_names=['amp_1', 'cen_1_x', 'wid_1_x', 'cen_1_y', 'wid_1_y',
                            'amp_2', 'cen_2_x', 'wid_2_x', 'cen_2_y', 'wid_2_y'])

# Performing the fit

result = gmodel.fit(data_2D, x=X, y=Y,
                    amp_1=650, cen_1_x=600, wid_1_x=60, cen_1_y=300, wid_1_y=10,
                    amp_2=160, cen_2_x=330, wid_2_x=40, cen_2_y=230, wid_2_y=10)   # this is the object returned by the fit operation

print(result.fit_report())


#Plot the result of the initial/best fit and comparison with the raw data

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')       
#ax.plot_surface(X,Y, result.init_fit, color='royalblue', antialiased=False)
ax.plot_surface(X, Y, -data_2D, color='red', antialiased=True)         # raw data
ax.plot_surface(X, Y, result.best_fit, color='royalblue', antialiased=False)   

plt.show()

[[Model]]
    Model(double_gaussian)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 67
    # data points      = 1562500
    # variables        = 10
    chi-square         = 10328956.6
    reduced chi-square = 6.61057451
    Akaike info crit   = 2951057.76
    Bayesian info crit = 2951180.38
    R-squared          = 0.99367326
[[Variables]]
    amp_1:    641.117987 +/- 0.05917327 (0.01%) (init = 650)
    cen_1_x:  604.573625 +/- 0.00682357 (0.00%) (init = 600)
    wid_1_x:  73.9338503 +/- 0.00682407 (0.01%) (init = 60)
    cen_1_y:  302.305265 +/- 0.00150060 (0.00%) (init = 300)
    wid_1_y:  16.2590589 +/- 0.00150077 (0.01%) (init = 10)
    amp_2:    160.474162 +/- 0.06006112 (0.04%) (init = 160)
    cen_2_x:  330.831066 +/- 0.02730467 (0.01%) (init = 330)
    wid_2_x:  72.9597880 +/- 0.02730666 (0.04%) (init = 40)
    cen_2_y:  235.065800 +/- 0.00598536 (0.00%) (init = 230)
    wid_2_y:  15.9932001 +/- 0.00598599 (0.04%) (init = 10)
[[Correlations]] (unre

In [12]:
# calculating the distance between the two gaussian lobes

distance = sqrt( (result.params['cen_2_x'].value - result.params['cen_1_x'].value)**2 \
                + (result.params['cen_2_y'].value - result.params['cen_1_y'].value)**2 )

print(distance)

281.8796449000023
